<a href="https://colab.research.google.com/github/roanfersa/detectCNH/blob/main/detectCNH.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **INSTALANDO AS FERRAMENTAS QUE SERÃO UTILIZADAS**

In [7]:
!pip install -q google-generativeai requests PyPDF2 pdf2image
!apt-get install -y -q poppler-utils > /dev/null 2>&1





#**IMPORTANDO TODAS AS BIBLIOTECAS QUE SERÃO UTILIZADAS**

In [8]:
import pathlib
import re
import PyPDF2
import textwrap
import requests
import PIL.Image
import json
import google.generativeai as genai
from google.colab import userdata, files
from IPython.display import display
from IPython.display import Markdown
from pathlib import Path
from pdf2image import convert_from_path




# **DEFININDO FUNÇÕES**

In [11]:
def to_markdown(text):
  text = text.replace('•', '  *')
  return Markdown(textwrap.indent(text, '> ', predicate=lambda _: True))

def verificar_cpf_usuario(nome, x_cpf_usuario):
    # Verificar o formato do CPF usando uma expressão regular para o formato XXX.XXX.XXX-XX
    formato_correto = re.match(r'^\d{3}\.\d{3}\.\d{3}-\d{2}$', x_cpf_usuario)
    # Extrair apenas os dígitos para verificar a contagem
    digitos_cpf = re.sub(r'\D', '', x_cpf_usuario)  # Remove tudo que não é dígito

    # Verificar ambas as condições: formato correto e contagem de dígitos
    if formato_correto and len(digitos_cpf) == 11:
        return (True, f"\nObrigado por fornecer suas informações, {nome}. Seu CPF é válido. Estamos processando a validação de autenticidade da sua CNH com base no CPF: {x_cpf_usuario}.")
    else:
        return (False, "Formato de CPF inválido. Por favor, insira o CPF no formato XXX.XXX.XXX-XX e certifique-se de que contém exatamente 11 dígitos numéricos.")


# **CONFIGURANDO O MODELO DO GOOGLE API GEMINI**

In [12]:
api_key = userdata.get('SECRET_KEY')
genai.configure(api_key=api_key)

In [13]:
# Configurações do modelo
generation_config = {
    "temperature": 1,
    "top_p": 1,
    "top_k": 0,
}

safety_settings ={
    'HATE': 'BLOCK_NONE',
    'HARASSMENT': 'BLOCK_NONE',
    'SEXUAL' : 'BLOCK_NONE',
    'DANGEROUS' : 'BLOCK_NONE'
}

# Instrução para o sistema
system_instruction = "Extraia dados críticos da imagem da CNH para verificação de autenticidade"

# Configurar o modelo de geração
model = genai.GenerativeModel(model_name="gemini-pro-vision",
                              generation_config=generation_config,
                              safety_settings=safety_settings)

# **SOLICITANDO OS DADOS DO USUÁRIO PARA INICIAR A VERIFICAÇÃO**

In [14]:
print("Por favor, forneça as informações solicitadas abaixo para realizar a validação de autenticidade da CNH.")

# Solicitar o nome do usuário
nome = input("Digite seu nome: ")

while True:
    # Solicitar o CPF do usuário e armazená-lo na variável x_cpf_usuario
    x_cpf_usuario = input("Digite seu CPF no formato XXX.XXX.XXX-XX: ")

    # Chamar a função verificar_cpf_usuario e imprimir o resultado
    validado, mensagem = verificar_cpf_usuario(nome, x_cpf_usuario)
    print(mensagem)

    # Sair do loop se o CPF for válido
    if validado:
        break

Por favor, forneça as informações solicitadas abaixo para realizar a validação de autenticidade da CNH.
Digite seu nome: ROAN
Digite seu CPF no formato XXX.XXX.XXX-XX: 179.660.217-50

Obrigado por fornecer suas informações, ROAN. Seu CPF é válido. Estamos processando a validação de autenticidade da sua CNH com base no CPF: 179.660.217-50.


# **SOLICITANDO O UPLOAD DO ARQUIVO CONTENDO A CNH**


---

```
O upload da carteira CNH pode ser uma imagem ou arquivo.pdf
```

Caso não tenha um CNH para realizar a verificação utilize os arquivos no github "cnh_falsa_pinterest.jpg" ou "cnh_falsa_pinterest.pdf"

Basta que você baixe-os para seu pc, e faça o upload aqui.

In [15]:
print("Por favor, faça o upload de uma imagem clara da Carteira Nacional de Habilitação (CNH) que você queira verificar abaixo.")
uploaded = files.upload()  # Permite ao usuário fazer upload da imagem da CNH
file_name = next(iter(uploaded))  # Pega o nome do arquivo
file_path = '/content/' + file_name  # Monta o caminho completo do arquivo

# Converter string para objeto Path para poder usar .suffix
file_obt_path = Path(file_path)

if file_obt_path.suffix.lower() == '.pdf':
        # Se for um PDF, converter a primeira página em imagem
        pagina01 = convert_from_path(file_path, last_page=1)[0]

        # Salvar a primeira página como imagem
        image_path = file_obt_path.parent / 'arquivo_1.jpg'
        pagina01.save(image_path, 'JPEG')
        print(f'Primeira página salva como imagem em: {image_path}')

        # Abrir imagem com PIL e adicionar ao array de imagens abertas
        open_image = PIL.Image.open(image_path)

else:
    # Se for uma imagem, apenas abri-la com PIL e adicionar à lista
    open_image = PIL.Image.open(file_path)


Por favor, faça o upload de uma imagem clara da Carteira Nacional de Habilitação (CNH) que você queira verificar abaixo.


Saving Foto CNH _ PDF.jpeg to Foto CNH _ PDF.jpeg


# **VERIFICANDO SE O ARQUIVO ENVIADO CORRESPONDE A UMA CNH USANDO O GEMINI VISION PRO**

p.s: para garantir que o arquivo realmente se trata de uma CNH.

In [16]:
response = model.generate_content(["Responda se a imagem se trata de uma Carteira CNH Brasileira. Responda 'sim' ou 'não'.", open_image])
response = response.text
if response.strip().lower() == 'não':
  print("Isso não é uma CNH, tente novamente")

else:
  print("Esta foto é de uma CNH, podemos prosseguir com a análise.")

Esta foto é de uma CNH, podemos prosseguir com a análise.


# **Extração dos Dados da CNH e FORMATAÇÃO PARA ENVIO AO DENATRAN**

In [22]:
extrair_info = model.generate_content(["Capture na foto as informações: CPF, Número de Registro", open_image])

extrair_info = extrair_info.text

# Usar expressões regulares para encontrar o CPF e o Número de Registro
match_cpf = re.search(r'CPF: (\d{3}\.\d{3}\.\d{3}-\d{2})', extrair_info)
match_numero_registro = re.search(r'Número de Registro: (\d+)', extrair_info)

# Armazenar os valores encontrados nas variáveis
cpf_cnh = match_cpf.group(1) if match_cpf else 'CPF não encontrado'
numero_registro_cnh = match_numero_registro.group(1) if match_numero_registro else 'Número de Registro não encontrado'

print("DADOS EXTRAÍDOS E FORMATADOS COM SUCESSO:")
print(f'CPF –> {cpf_cnh}')
print(f'Número de Registro –> {numero_registro_cnh}')

DADOS EXTRAÍDOS E FORMATADOS COM SUCESSO:
CPF –> 098.352.184-04
Número de Registro –> 05346893150


# **CONFIGURAÇÃO DA API DO DENATRAN E VALIDAÇÃO DA CNH**

Para saber sobre a API do DENATRAN


*   Baixe a documentação da API no Github do Projeto -> Link do Github
*   Ou confira as informações no site oficial da API do DENATRAN -> [WSDenatran API - Veículos, Condutores e Infrações](https://www.gov.br/conecta/catalogo/apis/wsdenatran)

Para usar a API do Denatran substitua no código abaixo abaixo:

```
# 1. O caminho do seu certificado digital
# 2. O caminho da sua chave privada
```



**Atenção: –** Caso você não tenha os elementos acima ou não tenha conseguido acesso a API do DENATRAN por algum motivo. Faça o seguinte:

```
# 1. Vá no Github do Projeto
# 2. Baixe o arquivo 'return_denatran_404.json''
# 3. Vá na secção files aqui no google colab, e faça o upload do arquivo.
# 4. Pule este bloco de código, e continue de lá.
```

*p.s: o arquivo return_denatran_404.json contem o retorno da API do DENATRAN para a imagem de exemplo "cnh_falsa_pinterest.jpg" e "cnh_falsa_pinterest.pdf".*






---



In [ ]:
# URL da API do DENATRAN
url = f'https://renavam.denatran.serpro.gov.br/v1/condutores/cpf/{cpf_cnh}/registroCnh/{numero_registro_cnh}'

# Cabeçalhos
headers = {
    'x-cpf-usuario': x_cpf_usuario
}

# Certificado digital (ajuste o caminho para o seu certificado)
certificado = ('caminho_para_certificado.crt', 'caminho_para_chave_privada.key')

# Fazer a requisição
response = requests.get(url, headers=headers, cert=certificado)

# Verificar o status da resposta
if response.status_code == 200:
    # Requisição bem-sucedida (OK)
    print("Sucesso! Dados do condutor:", response.json())
elif response.status_code == 401:
    # Não autorizado (Unauthorized)
    print("Erro: Não autorizado. Verifique seu certificado digital.")
elif response.status_code == 404:
    # Não encontrado (Not Found)
    print("Erro: Condutor não encontrado na base de dados do Denatran.")
elif response.status_code == 500:
    # Erro no servidor (Internal Server Error)
    print("Erro: Ocorreu um erro no servidor do Denatran.")
else:
    # Outros erros
    print(f"Erro na requisição: {response.status_code} - {response.text}")

# **VALIDAÇÃO DA CNH COM JSON LOCAL E VALIDAÇÃO DA CNH**

In [38]:
# Abrir e ler o arquivo JSON local
with open("/content/return_denatran_404.json", "r") as file:       # with open("/content/return_denatran_200.json", "r") as file: #-> use este código para rodar o return_denatran_200.json que é um caso de CNH Válida
    response_json = json.load(file)

# Simular o código de status (você pode ajustar conforme necessário)
status_code = response_json.get('returnCode', 200)

# Verificar o status da resposta
if status_code == 200:
    # Requisição bem-sucedida (OK)
    print(" Essa CNH é autêntica! Segue abaixo um resumo do condutor: ")
elif status_code == 401:
    # Não autorizado (Unauthorized)
    print("Erro: Não autorizado. Verifique seu certificado digital.")
elif status_code == 404:
    # Não encontrado (Not Found)
    print("CNH não autêntica:", response_json['message'])
elif status_code == 500:
    # Erro no servidor (Internal Server Error)
    print("Erro: Ocorreu um erro no servidor do Denatran.")
else:
    # Outros erros
    print(f"Erro na requisição: {status_code} - {response_json.get('message', 'Erro desconhecido')}")

if status_code == 200:
  # Configurar o modelo com o Gemini 1.0 PRO
    model = genai.GenerativeModel(model_name="gemini-pro",
                              generation_config=generation_config,
                              safety_settings=safety_settings)
  # Requisição ao Denatran bem-sucedida
data_condutor = response_json

# Dados do condutor já estão no dicionário data_condutor
json_text = json.dumps(data_condutor, indent=4)  # Convertendo o dicionário em string JSON formatada

prompt_gemini = f"""
    Você é um assistente de IA que gera resumos de dados de condutores.

    Aqui está um JSON contendo dados de um condutor do Denatran:
    {json_text}

    Gere um resumo informativo e objetivo contendo as informações mais relevantes sobre o condutor.
    """

    # Enviar o prompt diretamente
response_gemini = model.generate_content({prompt_gemini})

    # Exibindo o resultado
to_markdown(response_gemini.text)

CNH não autêntica: Condutor não encontrado na base de dados do Denatran.


> Não foi possível localizar o condutor na base de dados do Denatran.